In [1]:
import torch
from thesis_code.models.gans import LitHAGAN

/Users/rasmuspallisgaard/Developer/skole/thesis_code/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def tree_key_map(f, tree):
    if isinstance(tree, dict):
        return {f(k): tree_key_map(f, v) for k, v in tree.items()}
    elif isinstance(tree, list):
        return [tree_key_map(f, v) for v in tree]
    else:
        return tree

In [3]:
model = torch.load("/Users/rasmuspallisgaard/Downloads/results/pretraining/models/hagan-l5-1-no-zerosliced.ckpt", map_location="cpu")
author_generator = torch.load("/Users/rasmuspallisgaard/Downloads/GSP_HA_GAN_pretrained/G_iter80000.pth")

In [4]:
model["state_dict"].keys()

odict_keys(['G.fc1.weight', 'G.fc1.bias', 'G.tp_conv1.weight', 'G.tp_conv1.bias', 'G.bn1.weight', 'G.bn1.bias', 'G.tp_conv2.weight', 'G.tp_conv2.bias', 'G.bn2.weight', 'G.bn2.bias', 'G.tp_conv3.weight', 'G.tp_conv3.bias', 'G.bn3.weight', 'G.bn3.bias', 'G.tp_conv4.weight', 'G.tp_conv4.bias', 'G.bn4.weight', 'G.bn4.bias', 'G.tp_conv5.weight', 'G.tp_conv5.bias', 'G.bn5.weight', 'G.bn5.bias', 'G.tp_conv6.weight', 'G.tp_conv6.bias', 'G.bn6.weight', 'G.bn6.bias', 'G.tp_conv7.weight', 'G.tp_conv7.bias', 'G.sub_G.tp_conv1.weight', 'G.sub_G.tp_conv1.bias', 'G.sub_G.bn1.weight', 'G.sub_G.bn1.bias', 'G.sub_G.tp_conv2.weight', 'G.sub_G.tp_conv2.bias', 'G.sub_G.bn2.weight', 'G.sub_G.bn2.bias', 'G.sub_G.tp_conv3.weight', 'G.sub_G.tp_conv3.bias', 'D.conv1.weight', 'D.conv1.bias', 'D.conv1.u', 'D.conv1.v', 'D.conv2.weight', 'D.conv2.bias', 'D.conv2.u', 'D.conv2.v', 'D.conv3.weight', 'D.conv3.bias', 'D.conv3.u', 'D.conv3.v', 'D.conv4.weight', 'D.conv4.bias', 'D.conv4.u', 'D.conv4.v', 'D.conv5.weight', 

In [5]:
def fix_key(key):
    if key.startswith("module.sub_G."):
        return "G.sub_G." + key[len("module.sub_G."):]
    elif key.startswith("module."):
        return "G." + key[len("module."):]
    return key

author_generator_fixed = tree_key_map(fix_key, author_generator["model"])
# jax.tree.map(lambda x: x.shape, author_generator_fixed)
author_generator_fixed.keys()

dict_keys(['G.fc1.weight', 'G.fc1.bias', 'G.tp_conv1.weight', 'G.tp_conv1.bias', 'G.bn1.weight', 'G.bn1.bias', 'G.tp_conv2.weight', 'G.tp_conv2.bias', 'G.bn2.weight', 'G.bn2.bias', 'G.tp_conv3.weight', 'G.tp_conv3.bias', 'G.bn3.weight', 'G.bn3.bias', 'G.tp_conv4.weight', 'G.tp_conv4.bias', 'G.bn4.weight', 'G.bn4.bias', 'G.tp_conv5.weight', 'G.tp_conv5.bias', 'G.bn5.weight', 'G.bn5.bias', 'G.tp_conv6.weight', 'G.tp_conv6.bias', 'G.bn6.weight', 'G.bn6.bias', 'G.tp_conv7.weight', 'G.tp_conv7.bias', 'G.sub_G.tp_conv1.weight', 'G.sub_G.tp_conv1.bias', 'G.sub_G.bn1.weight', 'G.sub_G.bn1.bias', 'G.sub_G.tp_conv2.weight', 'G.sub_G.tp_conv2.bias', 'G.sub_G.bn2.weight', 'G.sub_G.bn2.bias', 'G.sub_G.tp_conv3.weight', 'G.sub_G.tp_conv3.bias'])

In [6]:
for k, v in author_generator_fixed.items():
    if k in model["state_dict"]:
        model["state_dict"][k] = v
    else:
        raise ValueError(f"Key {k} not found in model state dict")
# torch.save(model, "/Users/rasmuspallisgaard/Downloads/hagan-from-authors.ckpt")

In [7]:
# Create new state dict with renamed keys
new_state_dict = {}
for key, value in model["state_dict"].items():
    if key.endswith(".u"):
        new_key = key[:-2] + ".u0" 
    elif key.endswith(".v"):
        new_key = key[:-2] + ".sv0"
    else:
        new_key = key
    new_state_dict[new_key] = value

# Update model state dict with renamed keys
# model["state_dict"] = new_state_dict

In [8]:
model["state_dict"].keys()

odict_keys(['G.fc1.weight', 'G.fc1.bias', 'G.tp_conv1.weight', 'G.tp_conv1.bias', 'G.bn1.weight', 'G.bn1.bias', 'G.tp_conv2.weight', 'G.tp_conv2.bias', 'G.bn2.weight', 'G.bn2.bias', 'G.tp_conv3.weight', 'G.tp_conv3.bias', 'G.bn3.weight', 'G.bn3.bias', 'G.tp_conv4.weight', 'G.tp_conv4.bias', 'G.bn4.weight', 'G.bn4.bias', 'G.tp_conv5.weight', 'G.tp_conv5.bias', 'G.bn5.weight', 'G.bn5.bias', 'G.tp_conv6.weight', 'G.tp_conv6.bias', 'G.bn6.weight', 'G.bn6.bias', 'G.tp_conv7.weight', 'G.tp_conv7.bias', 'G.sub_G.tp_conv1.weight', 'G.sub_G.tp_conv1.bias', 'G.sub_G.bn1.weight', 'G.sub_G.bn1.bias', 'G.sub_G.tp_conv2.weight', 'G.sub_G.tp_conv2.bias', 'G.sub_G.bn2.weight', 'G.sub_G.bn2.bias', 'G.sub_G.tp_conv3.weight', 'G.sub_G.tp_conv3.bias', 'D.conv1.weight', 'D.conv1.bias', 'D.conv1.u', 'D.conv1.v', 'D.conv2.weight', 'D.conv2.bias', 'D.conv2.u', 'D.conv2.v', 'D.conv3.weight', 'D.conv3.bias', 'D.conv3.u', 'D.conv3.v', 'D.conv4.weight', 'D.conv4.bias', 'D.conv4.u', 'D.conv4.v', 'D.conv5.weight', 

In [9]:
torch.save(model, "/Users/rasmuspallisgaard/Downloads/hagan-from-authors.ckpt")

## Test it out

In [10]:
test_model = LitHAGAN.load_from_checkpoint("/Users/rasmuspallisgaard/Downloads/hagan-from-authors.ckpt", latent_dim=1024, lambda_1=5, lambda_2=5, use_dp_safe=True)

In [11]:
author_discriminator = torch.load("/Users/rasmuspallisgaard/Downloads/GSP_HA_GAN_pretrained/D_iter80000.pth")

In [12]:
author_discriminator["model"].keys()

odict_keys(['module.conv1.weight', 'module.conv1.bias', 'module.conv1.u0', 'module.conv1.sv0', 'module.conv2.weight', 'module.conv2.bias', 'module.conv2.u0', 'module.conv2.sv0', 'module.conv3.weight', 'module.conv3.bias', 'module.conv3.u0', 'module.conv3.sv0', 'module.conv4.weight', 'module.conv4.bias', 'module.conv4.u0', 'module.conv4.sv0', 'module.conv5.weight', 'module.conv5.bias', 'module.conv5.u0', 'module.conv5.sv0', 'module.conv6.weight', 'module.conv6.bias', 'module.conv6.u0', 'module.conv6.sv0', 'module.conv7.weight', 'module.conv7.bias', 'module.conv7.u0', 'module.conv7.sv0', 'module.fc1.weight', 'module.fc1.bias', 'module.fc1.u0', 'module.fc1.sv0', 'module.fc2.weight', 'module.fc2.bias', 'module.fc2.u0', 'module.fc2.sv0', 'module.sub_D.conv1.weight', 'module.sub_D.conv1.bias', 'module.sub_D.conv1.u0', 'module.sub_D.conv1.sv0', 'module.sub_D.conv2.weight', 'module.sub_D.conv2.bias', 'module.sub_D.conv2.u0', 'module.sub_D.conv2.sv0', 'module.sub_D.conv3.weight', 'module.sub_D.